# Lua scripting

Redis has an embedded sandboxed Lua engine that can be invoked for executing user scripts. Scripts, executed by the server, are atomic and have full access to the data.

## What is Lua

TL;DR it is a scripting language.

According to [https://www.lua.org/about.html](https://www.lua.org/about.html):

> Lua is a powerful, efficient, lightweight, embeddable scripting language. It supports procedural programming, object-oriented programming, functional programming, data-driven programming, and data description.
>
> Lua combines simple procedural syntax with powerful data description constructs based on associative arrays and extensible semantics. Lua is dynamically typed, runs by interpreting bytecode with a register-based virtual machine, and has automatic memory management with incremental garbage collection, making it ideal for configuration, scripting, and rapid prototyping.

### Redis Lua Sandbox

As of v2.6, Redis embeds a Lua v5.1 engine for running user scripts. The engine is sandboxed, in the sense the it exposes a subset of Lua's full functionality, for security purposes.

The restrictions put by the sandbox are:

* Only [local](https://www.lua.org/manual/5.1/manual.html#2.4.7) declarations are permitted. Attempting to create global variables, or accessing non existing ones, generates an error. 
* External libraries can't be used, the `require()` function is disabled
* Standard built-in libraries that are included:  `base`, `string`, `table` and [`math`](https://www.lua.org/manual/5.1/manual.html#5.6)
* Standard builit-in libraries that are disabled: `io`, `os` and `debug`
* Additional bundled libraries:
  * `bit` - bit manipulation
  * `struct` - encode/decode C-like structures
  * `cjson` - encode/decode JSON
  * `cmsgpack` -encode/decode MessagePack
  * `redis` - API to the Redis server
* A couple of potentially harmful functions are disabled: [`dofile`](https://www.lua.org/manual/5.1/manual.html#pdf-dofile) and [`loadfile`](https://www.lua.org/manual/5.1/manual.html#pdf-dofile)

Despite all these restrictions, Redis' Lua is extremely powerful and allows doing wonderful things :)

## Lua 101: Hello, World!

<pre><code>

-- The "Hello, World!" program, a Redis Lua example
local message = 'Hello, World!'
return message

</code></pre>

The example above is in Lua. The first line, beginning with a double dash, is a comment. In the second line a local variable named `message`  is declared and assigned with a string value. In the last line, the script ends and returns the variable's contents to the caller.

> **Note**
>
> Although [`print`](https://www.lua.org/manual/5.1/manual.html#pdf-print) is available, its output is directed to the Redis server's stdout so it is practically useless in most contexts.

## Running server-side Lua scripts

### One-off scripts

You can run one-off scripts - e.g. for development purposes - in Redis with the [`EVAL`](https://redis.io/commands/eval) command:

In [ ]:
import redis

# example connection parameters 
config = {
    "host": "redis",
    "port": 6379
}

r = redis.StrictRedis(**config)
script = """
-- The "Hello, World!" program, a Redis Lua example
local message = 'Hello, World!'
return message
"""

reply = r.eval(script, 0)
print ('Reply: {}'.format(reply))

When `eval()` is invoked, the script itself is sent to Redis for compilation into byte code and execution. The "Hello, World!" example requires no inputs, so the second argument to `eval()` is 0, which is the number of keys that the script gets as parameters.

### Cached scripts

Redis caches the byte code of every script it executes in order to avoid re-compiling it in subsequent calls. To avoid repeatedly sending the same script over the network, you can load a script directly into Redis' scripts' cache and use an identifier to invoke it. Cached scripts offer two major performance gains:

* The script's payload is sent only once to the server, reducing the amount of traffic needed for each subsequent invocation
* The script is parsed and compiled to byte code only once, so subsequent executions are immediate

Loading a script to the cache is achieved with the [`SCRIPT LOAD`](https://redis.io/commands/script-load) command. It accepts the script's payload and returns, if compilation is successful, a unique identifier for the script that is its SHA1 sum. To execute a cached script, use the [`EVALSHA`](https://redis.io/commands/evalsha) command.

In [ ]:
# the script can hardcoded or loaded from somewhere else
sha = r.script_load(script)
reply = r.evalsha(sha, 0)

print ('SHA1: {}, Reply: {}'.format(sha, reply))

The scripts cache is not persisted. That means that when Redis is restarted, all cached scripts need to be reloaded by the client. The [`SCRIPT EXISTS`](https://redis.io/commands/script-exists) command can be used to verify that a given SHA1 sum is in the cache.

### Long running scripts

Script execution is atomic, so the server blocks all other requests while a script is run. However, unlike the core Redis commands, scripts' execution times can be arbitrarily long. Redis has a configurable threshold setting, the `lua-time-limit` directive, that expresses the maximum time in milliseconds before a running script is deemed as long running:

In [ ]:
print (r.config_get('lua-time-limit'))

Once a script has crossed the long running threshold and until it finishes execution, all requests to the server will be  declined with the following error message:

`BUSY Redis is busy running a script. You can only call SCRIPT KILL or SHUTDOWN NOSAVE.`

To abort a long running script, use [`SCRIPT KILL`](https://redis.io/commands/script-kill). However, killing a script is not allowed if the script modified any data. In such cases, only an immediate shutdown is possible.

### Scripts cache considerations

Every script executed in Redis, whether explicitly cached or not, is stored in the cache. This allows Redis to skip the compilation of one-off scripts that are sent multiple times.

While every script is added to the cache, none are removed by Redis. The cache's current size can be obtained with the [`INFO`](https://redis.io/commands/info) command:

In [ ]:
print (r.info()['used_memory_lua'])

Redis provides a single command for managing the script cache - [`SCRIPT FLUSH`](https://redis.io/commands/script-flush) - that flushes the entire cache. Because the script cache is not persisted, restarting the server also clears it.

## Lua 102: data types

Lua has eight [data types](https://www.lua.org/manual/5.1/manual.html#2.2):

1. **Nil**: can only have the value `nil` (evaluates to `false` in conditions)
2. **Boolean**: can be either `true` or `false`
3. **Number**: 32-bit, double-precision floating point, real numbers (integers as well)
4. **String**: 8-bit, binary-safe strings
5. **Function object**: for functions
6. **Userdata object**: used for binding with C (irrelevant in the Redis scripting context)
7. **Thread object**: used for implementing coroutines
8. **Table object**: an associative array

To find out the type of a given expression or variable, use Lua's [`type()`](https://www.lua.org/manual/5.1/manual.html#pdf-type) function.

### The Lua table data type

Arguably the most useful is the Lua table data type. It is an associative array, that can store key-value pairs much like a Python dictionary or a Redis Hash. The following Lua code declares a local variable and initializes it with an empty table.

`local emptyTable = {}`

To create an initialized associative array, the following syntax is used:

`local associativeArray = { ['key1'] = 'value1', [42] = true }`

Accessing an element in an associative array is either by using the bracketed notation, i.e. `associativeArray[42]`, or with an object-like dotted canonical form, i.e. `associativeArray.key1`. Setting element in an array to the value `nil` is the equivalent of deleting it. Adding key-value pairs to the arrays is as simple as:

`associativeArray['new'] = 'some value'`

That said, Lua tables can also double as 1-based indexed arrays:

`local indexedArray = { 1, 'foo', false }`

In which case access is via bracketed index notation, i.e. `indexedArray[1]`. To obtain the current length of an indexed array (i.e. the index of the last element), use `#indexedArray`. To append to the indexed array use the following syntax:

`indexedArray[#indexedArray+1] = 'appended value'`

or use [`table.insert()`](https://www.lua.org/pil/19.2.html):

`table.insert(indexedArray, 'appended value')`

To remove an element for the array, call `table.remove()`.

## Script parameters

### Passing parameters

Some scripts require input parameters when called. With Redis' Lua scripts there's a clear distinction between two types of input:

1. Key names - the names of **all** keys touched by the script
2. Arguments - anything that isn't a key's name

**Important:** explicitly stating all key names when executing a script is a strong recommendation. While Redis does not enforce it, not doing so is a sure path to future scalability challenges.

A script's parameters, first key names and then the rest of the arguments, are passed with Redis' `EVAL` (and `EVALSHA`) and are preceded by the number of keys present. For example, to call a script that accepts a single key name ("foo") and two additional arguments ("bar" and "baz"), the following syntax is used:

`EVAL "-- some Lua code here" 1 foo bar baz`

This is the same with redis-py's `eval()` (and `evalsha()`):

In [ ]:
r.eval('-- some Lua code here', 1, 'foo', 'bar', 'baz')

### Parsing parameters

The parameters passed to a Redis Lua script are available to it via two Lua tables: `KEYS` and `ARGV`. The first table is populated with all key names, i.e. the first arguments up to the count specified, whereas the second table contains the rest. Both tables are 1-base indexed arrays. The script can access the contents of a Lua table by index using the bracketed array notation:

In [ ]:
scriptParams = """
--[[
  Note that this script also demonstrates:
  * a multi-line comment
  * Lua's varlist assignment
  * String concatanation
]]--
local key = KEYS[1]
local arg1, arg2 = ARGV[1], ARGV[2]

local reply = 'Called with key "' .. key .. '" and args "' .. arg1 .. '" and "' .. arg2 .. '"'
return reply
"""

print (r.eval(scriptParams, 1, 'foo', 'bar', 'baz'))

## Calling Redis from a Lua script running in Redis

Accessing Redis from a server-side Lua script is done with the [`redis.call()`]() function. It can be used to call (almost) any Redis command from the script, and returns the reply from that command to the script:

In [ ]:
scriptPing = """
    local reply = redis.call('PING')
    return reply
"""

print (r.eval(scriptPing, 0))

A script can call variadic Redis commands with any number of arguments. Lua's [`unpack()`](https://www.lua.org/manual/5.1/manual.html#pdf-unpack) function comes in very handy, as it allows converting tables (lists) into arguments (identical to the splat operator). As an example, here's a Lua script that preforms the equivalent of Redis' `SADD`, but on two sets. Each call to `SADD` is done with the entire list of the script's arguments:

In [ ]:
shaMultiSadd = r.script_load( """
    local set1, set2 = KEYS[1], KEYS[2]
    redis.call('SADD', set1, unpack(ARGV))
    redis.call('SADD', set2, unpack(ARGV))
    return 'OK'
""" )

print (r.evalsha(shaMultiSadd, 2, 'characters', 'hobbits', 'Bilbo', 'Frodo'))
print (r.evalsha(shaMultiSadd, 2, 'characters', 'wizards', 'Gandalf', 'Saruman'))

### Error handling

In case of an error, `redis.call()` will raise a Lua error that will be sent back to the client. To trap errors and handle them in the script's code, use `redis.pcall()` instead.

### Returning from a script

A script returns once it reaches its end, or upon encountering the [`return`](https://www.lua.org/manual/5.1/manual.html#2.4.4) Lua statement. A script that ends without explicitly calling `return`, will return a Redis `nil`. If `return` is called, the value passed to it will be returned to the script's caller.

The built-in `redis` library has two helper functions to wrap replies:

* `redis.reply_status(status_string)` returns a status message
* `redis.reply_error(error_string)` returns an message

### Conversions between data types

Return values from `redis.call()` and `redis.pcall()` are converted into Lua data types. Similarly, Lua data types are converted into the Redis protocol when calling a Redis command and when a Lua script returns a value.

This conversion between data types is designed in a way that if a Redis type is converted into a Lua type, and then the result is converted back into a Redis type, the result is the same as the initial value. In other words there is a one-to-one conversion between Lua and Redis types.

#### Redis to Lua

| The Redis                           | Is converted to a Lua                 |
| ----------------------------------- | ------------------------------------- |
| Integer reply                       | Number                                |
| Bulk string reply                   | String                                |
| Multi bulk reply                    | Table, may be nested                  |
| Status reply                        | Table with a single field, `ok`, set  |
| Error reply                         | Table with a single field, `err`, set |
| Nil bulk and Nil multi bulk replies | Boolean `false`                       |

#### Lua to Redis

| The Lua data type               | Is converted to a Redis                  |
| ------------------------------- | ---------------------------------------- |
| Number                          | Integer reply                            |
| String                          | Bulk string reply                        |
| Table (indexed array)           | Multi bulk reply (truncated at the first nil in the array, if any) |
| Table with a single `ok` field  | Status reply                             |
| Table with a single `err` field | Error reply                              |
| Boolean `false`                 | Nil bulk reply                           |
| Boolean `true`                  | Integer reply with value of 1 (has no corresponding conversion) |

#### Conversion notes

- Lua has a single numerical type, Number, for both integers and floats. Always convert Lua numbers into integer replies, removing the decimal part of the number if any. If you want to return a float from Lua you should return it as a string, and convert a string reply to a number if reading a float value from Redis.
- There is [no simple way to have nils inside Lua arrays](http://www.lua.org/pil/19.1.html), this is a result of Lua table semantics, so when Redis converts a Lua array into Redis protocol the conversion is stopped if a nil is encountered.
- The conversion of Lua's associative arrays, i.e. tables that are not arrays/lists, is similarly non-trivial, so when Redis converts an associative Lua array into Redis protocol the conversion is stopped when the first key in the array is encountered (except for status and replies as described above).

## Lua 103: control structures

The [control structures](https://www.lua.org/manual/5.1/manual.html#2.4.4) **if**, **while**, and **repeat** have the usual meaning and familiar syntax.

* if statement: **if** exp **then** block {**elseif** exp **then** block} [else block] **end**
* while loop: **while** exp **do** block **end**
* repeat loop: **repeat** block **until** exp

Lua also has a **for** statement, in [two flavors](https://www.lua.org/manual/5.1/manual.html#2.4.5):

* Numeric for loop: **for** Name = exp , exp [ , exp] **do** block **end**
* Iterator for loop: **for** namelist **in** explist **do** block **end**

In the above, "block" is one or more Lua statements and "exp" is a logical expression.

> Note: logical expressions evaluate to Boolean `true` for all values except `false` and `nil`. This means that the number 0 also evaluates to `true`

Finally, **break** can be called for exiting the current loop.

## Scripting use cases

### Reducing client->server bandwidth

Scripts can be employed to effectively reduce the bandwidth required, and as a result the average latency, for communicating between the client and server in several manners.

First and foremost the script cache, as previously noted, serves this purpose by saving the need to repeatedly send the same script payload.

Secondly, because of Lua's control structures, further "compression" of a multiple commands into much shorter scripts is possible. Conditional and loop statements can be implemented and executed entirely server-side, similar RDBMSs' stored procedures.

For example, consider a function that calls Redis' `LPOP` multiple times in a pipeline:

In [ ]:
def pipelinedMultiLpop(key, count):
    p = r.pipeline(transaction=False)
    for _ in range(count):
        p.lpop(key)
    return p.execute()

Calling `pipelinedMultiLpop(100, 'somelist')` will result in the client sending the server the same command 100 times, namely `LPOP somelist`. Ignoring the network's overhead, this totals at:

In [ ]:
print ('pipelinedMultiLpop payload: {} bytes'.format(100 * len('LPOP somelist')))

The same functionality can be implemented with a Lua script:

In [ ]:
shaMultiLpop = r.script_load( """
    local key = KEYS[1]
    local count = tonumber(ARGV[1])
    local reply = {}
    for _ = 1, count do
        reply[#reply+1] = redis.call('LPOP', key)
    end
    return reply
""" )

def scriptedMultiLpop(key, count):
    return r.evalsha(shaMultiLpop, 1, key, count)

print ('initialization payload: {} bytes'.format(len(script)))
print ('scriptedMultiLpop payload: {} bytes'.format(len('EVALSHA {} 1 somelist 100'.format(sha))))

After "paying" the initial loading price once, the invocations only require a minimal amount of communication.

### Reducing server->client bandwidth

Lua scripts can contain (almost) any user-defined logic and enjoy the benefit of data locality. While there is a (CPU) price to every call to `redis.call()`, interaction from the embedded Lua engine is still order of magnitudes faster than with a remote client. Having the data also means that scripts can process it, effectively providing the means to bring the CPU to the data instead of the other way around.

Aggregation is classic example - assume that you're using Redis for storing a bunch of numbers. Perhaps you're using a Bitfield for storing these numbers in a most compact manner. Or maybe they're store in a Hash under different fields. Lets suppose however, for the example's sake, that these numbers are stored as scores in a Sorted Set:

In [ ]:
# Set up some data
keyname = 'bunchofnumbers'
count = 10
initdata = [i for i in range(count) for _ in range(2)]
with r.pipeline() as p:
    r.delete(keyname)
    r.zadd(keyname, *initdata)
    p.execute()

You could sum them up by fetching the contents of the Sorted Sets, scores included, to the client code and doing the math there:

In [ ]:
def clientSum(key):
    reply = r.zrange(key, 0, -1, withscores=True)
    total = sum([score for ele, score in reply])
    return total

print clientSum(keyname)

Despite its brevity and the fact that it only calls one Redis command, it is the Sorted Set's size that will determine `clientSum()`'s impact. The bigger the value, the more resources - server CPU and network - it will take to send the reply to the client.

The crucial point to notice in the aggregation example is that the client isn't actually interested in the raw data at all - it needs it only as input for the next stage of processing. Implementing a Lua script in its stead would prevent the need to send back what could be a considerable amount of data that's extremely short-lived:

In [ ]:
shaSum = r.script_load( """
    local total = 0
    local data = redis.call('ZRANGE', KEYS[1], 0, -1, 'WITHSCORES')
    while #data > 0 do
        -- ZRANGE replies with an array in which scores are at even indices
        table.remove(data, 1)
        total = total + tonumber(table.remove(data, 1))
    end
    return total
""" )

def scriptedSum(key):
    return r.evalsha(shaSum, 1, key)

print scriptedSum(keyname)

### Composing commands

Despite Redis' sizable and constanly-growing arsenal of commands, there are always bound to be commands that are missing from it. Common examples of "missing" commands are:

* Lack of variadic variants for some commands, e.g. `LPOP`
* No support for multiple keys in others, e.g. `SADD`
* Missing subcommands, e.g. having [`SET`](https://redis.io/commands/set)-like `EX|NX`  flags for [`HSET`](https://redis.io/commands/hset)
* Striving for perfect symmetry, i.e. why is there no `LPOPRPUSH`?

But even if these were to be resolved eventually, gaps would always be present because some "missing" commands are too specialized to be included in the core. Such commands are no less common, but each is usually tailored to a specific use case of a particular application.

As demonstrated above, Lua scripts are a (near) perfect vehicle for implementing custom commands and fill these gaps. Scripting allows composing core Redis commands in a programmatic fashion for manipulate the data in (nearly) every way. 

### Better transactions

Perhaps the most compelling use case for Lua scripts is that like (almost) every Redis command, script execution is atomic and blocking. While transactions allow for that as well, these are undeniably more cumbersome to use when optimistic concurrency control is involved.

Recall the `checkBalanceAndTransferAmount()` example:

In [ ]:
def checkBalanceAndTransferAmount(debit, credit, amount):
    amount = float(amount)
    debitkey = 'account:{}'.format(debit)
    creditkey = 'account:{}'.format(credit)
    fname = 'balance'
    
    while True:
        try:
            tx = r.pipeline()
            tx.watch(debitkey)
            balance = float(tx.hget(debitkey, fname))
            tx.multi()
            if balance >= amount:
                tx.hincrbyfloat(debitkey, fname, -amount)
                tx.hincrbyfloat(creditkey, fname, amount)
                return tx.execute()
            else:
                raise Exception('insufficent funds - time to get a job')
        except WatchError:
            # Reaching here means that the watched 'balance' value had changed,
            # so we can just retry or use any other backoff logic
            continue

Instead, the same can be achieved with a Lua script with the added bonus of ditching concurrency control altogether:

In [ ]:
from hashlib import sha1

def scriptedTransaction(debit, credit, amount):
    __script = """
        local debitkey, creditkey, amount, fname = 
            KEYS[1],
            KEYS[2],
            tonumber(ARGV[1]),
            'balance'
        local balance = tonumber(redis.call('HGET', debitkey, fname))
        if balance >= amount then
            redis.call('HINCRBYFLOAT', debitkey, fname, -amount)
            redis.call('HINCRBYFLOAT', creditkey, fname, amount)
            return redis.status_reply('OK')
        else
            return redis.error_reply('insufficent funds - call parents')
        end
    """
    __sha = sha1(__script).hexdigest()
    
    while True:
        try:
            r.evalsha(__sha, 2, 'account:{}'.format(debit), 'account:{}'.format(credit), amount)
        except redis.exceptions.NoScriptError:
            r.script_load(__script)
        else:
            break

## Scripts vis a vis data persistence and replication

### Data persistence

Because RDB files are snapshots of the data in memory, the modifications made by scripts are included in them by definition.

This is different, however, with append-only files as they are essentially logs of commands. In order to replay the logs and execute calls to `EVALSHA`, the AOF's preamble includes the scripts themselves as well.

### Full script replication

Not unlike AOF, the replication stream to slave Redis instances (shards) also consists of the actual scripts' payload. That means that by default, whether `EVAL`ed or `EVALSHA`ed, a script will be executed not only in the master but also on every one of its slaves. 

This design is in place primarily so that slaves can provide all data persistence capabilities and serve as eligible candidates for promotion in case of need (and thus need to synchronize other slaves). Another benefit of this design is, similarly to how scripts reduce the client->server bandwidth, in "compressing" the traffic between the master and its slaves.

### Non-deterministic scripts are not allowed

Replicating scripts imposes one prominent restriction: in order to ensure that a script has the same effects wherever it is executed, all operations that it consists of must be deterministic. This means that once a script performs an operation that returns a random results, subsequent attempts to perform write operations from it will be blocked.  Trying to violate this restriction will result in an error - consider the "random" Redis command [`TIME`](https://redis.io/commands/time) that returns the server's clock reading:

In [ ]:
# The following script will fail due to attempting to write after a random command
setCurrentTime = """
    local t = redis.call('TIME')
    redis.call('SET', KEYS[1], tostring(t[1]) .. '.' .. tostring(t[2]))
"""

try:
    r.eval(setCurrentTime, 1, 'now')
except redis.exceptions.ResponseError as e:
    print e

When a timestamp is needed in a writing script, you'll need to pass it an argument. There are, however, a few more Redis commands with non-deterministic behavior other than `TIME`: 

* [`LASTSAVE`](https://redis.io/commands/lastsave)
* [`PUBSUB`](https://redis.io/commands/pubsub)
* [`RANDOMKEY`](https://redis.io/commands/randomkey)
* [`SPOP`](https://redis.io/commands/spop)
* [`SRANDMEMBER`](https://redis.io/commands/srandmember)
* [`SCAN`](https://redis.io/commands/scan), as well as `HSCAN`, `SSCAN` and `ZSCAN`

That said, in this mode Redis uses a custom implementation of Lua's `math.random()`  and `math.randomseed()` to ensure it behaves consistently in all executions.

### Script effects replication

As of version 3.2, Redis supports another mode for replicating scripts. Instead of using the script's body, the calls to `redis.call()` can be persisted (in AOF) and/or replicated (to slaves) Redis commands inside a `MULTI/EXEC` transaction. When using this replication mode Redis essentially captures the effect of a script, hence the its name.

One nice thing about effects replication is that it allows the execution of write operations after non-deterministic commands. Because effects are recorded rather than the operations the generate them, the is no danger of inconsistency.

The other benefit of using effect replication is that it helps avoid repeating time-consuming computations. In some cases, where the script's logic is complex and/or the data is voluminous, execution time can become significant. And while that price has to be paid at least once, script replication dictates that it will be repeated in every AOF recovery and/or slave synchronization. But when a slave is being synchronized, let alone when recovery from persistent storage is carried out, time is the one thing you don't want to waste.

To switch from the default, script-based replication to effects replication, a script needs to call the `redis.replicate_commands()` library function before performing any write operation. 

### Selective replication of commands

Once effects replication is used, it is possible to further control the persistence and replication of commands. This is useful, for example, when temporary data is written while the script is running - resources can be saved by not replicating transient effects, only the final meaningful ones.

> Note: this functionality is advanced and powerful - it can easily do damage when misused

A script can control the replication of effects by calling the `redis.set_repl()` library function. The function accepts a single argument that can be one of the following:

* `redis.REPL_ALL` - this is the default behavior in which effects are replicated to slaves and AOF
* `redis.REPL_AOF` - effects are stored only in AOF
* `redis.REPL_SLAVE` - effects are replicated only to slaves
* `redis.REPL_NONE` - effects are not replicated

## Debugging Lua scripts

Redis features a built-in stepping Lua debugger that was added in version 3.2. For more information on using the Lua debugger with `redis-cli` and via UI, refer to:

- [`SCRIPT DEBUG`](https://redis.io/commands/script-debug) 
- [Redis Lua scripts debugger](https://redis.io/topics/ldb)
- [ZeroBrane Studio Plugin for Redis Lua Scripts](https://redislabs.com/blog/zerobrane-studio-plugin-for-redis-lua-scripts/)

## Scripting considerations

* Refrain from using `EVAL` for anything but development purposes - use parametrized cached scripts instead - or you scripts cache will need constant care
* Do not generate scripts programmatically, as this can not be cached and are always `EVAL`ed - see top consideration
* Adhere to explicitly declaring the keys touched by a script by passing them using the `KEYS` table. Specifically:
  * Do not use key names that are hardcoded in your script
  * Do not use key names that are programmatically generated by the script, e.g. by concatenating values
  * Do not use key names that are based on data read from values in the database (in other keys)
* Keep your scripts lean and fast - long-running scripts have an impact on performance
* Scripts are meant to be stateless, pure functions - the protection against globals is designed to help that. If you have to provide a state to the script, use parameters. If the state must be stored in the database, use a key.
* A script can not run another script - `redis.call()` will error on `EVAL` or `EVALSHA`, and there is no documented way to invoke a cached script from the code of another (though an undocumented way is to call `f_theotherscriptssha1sum()`)
* Client-blocking commands as well as some administrative commands can not be used with `redis.call()` - a proper error message is returned to the caller in such cases

## Further reading

* [Lua 5.1 manual](https://www.lua.org/manual/5.1/manual.html) and [tutorials directory](http://lua-users.org/wiki/TutorialDirectory)
* [Redis `EVAL` command](https://redis.io/commands/eval)